<center><h1>Lending Club Model

## Section 1 - Data cleaning
We will try to create a ML model that can predict the maximum approved loan amount from approved and declined loan data. We begin by importing data from 2007 - 2014 on approved and denied loan applications and selecting only those variables for which both data sets have recorded data.

The years 2007-2014 have been chosen as the maximum amount of data that my algorithm could handle in a reasonable amount of time. I make the assumption that the amount that could be requested has not significantly changed over those years, hence this data can be randomly shuffled into training and testing data.

In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline  
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
# from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.simplefilter(action='ignore')

In [2]:
approved_07_11 = pd.read_csv('/Users/michellehackl/Google_Drive/School/Minerva/Classes/2018:2019/CS156/09:30 A2 - Lending Club Assignment/approved_07_11.csv')
approved_12_13 = pd.read_csv('/Users/michellehackl/Google_Drive/School/Minerva/Classes/2018:2019/CS156/09:30 A2 - Lending Club Assignment/approved_12_13.csv')
approved_14 = pd.read_csv('/Users/michellehackl/Google_Drive/School/Minerva/Classes/2018:2019/CS156/09:30 A2 - Lending Club Assignment/approved_14.csv')

In [3]:
approved = pd.concat([approved_07_11, approved_12_13, approved_14])
print approved.shape

(466360, 145)


I have identified the State, Debt-to-Income Ratio, Employment Length, Policy Code, Zip Code, Loan Title and Date as the only variables consistent across both data sets. Date was left out, as we are making the assumption that the year at which a request was made should not influence the amount. It would be potentially interesting to include the month, as this might show interesting patterns in the data, however the date in the declined data frame is "Application Date", while the date in the approved data frame is "Approval Date", which, for all we know, could be months apart. Hence it might be harder to identify meaningful trends and as such I instead chose to exclude the dates.

---
I included a short section in the appendix on potential processing steps for Loan Title and reasons as to why I chose not to include it.

In [4]:
approved = approved[['addr_state', 'dti', 'emp_length', 'loan_amnt', 'policy_code',
                    'title', 'zip_code']]

names = {'addr_state': 'State', 'dti': 'Debt-To-Income Ratio',
         'emp_length': 'Employment Length', 'loan_amnt': 'Amount Requested',
         'policy_code': 'Policy Code', 'title': 'Loan Title',
         'zip_code': 'Zip Code'}
approved.rename(index=str, columns=names, inplace=True)
approved.head(3)

,State,Debt-To-Income Ratio,Employment Length,Amount Requested,Policy Code,Loan Title,Zip Code
0,AZ,27.65,10+ years,5000.0,1.0,Computer,860xx
1,GA,1.00,< 1 year,2500.0,1.0,bike,309xx
2,IL,8.72,10+ years,2400.0,1.0,real estate business,606xx


In [5]:
declined_07_12 = pd.read_csv('/Users/michellehackl/Google_Drive/School/Minerva/Classes/2018:2019/CS156/09:30 A2 - Lending Club Assignment/declined_07_12.csv')
declined_13_14 = pd.read_csv('/Users/michellehackl/Google_Drive/School/Minerva/Classes/2018:2019/CS156/09:30 A2 - Lending Club Assignment/declined_13_14.csv')

In [6]:
declined = pd.concat([declined_07_12, declined_13_14])
declined = declined.drop(['Application Date', 'Risk_Score'], axis=1)
print declined.shape
declined.head(3)

(1804065, 7)


,Amount Requested,Loan Title,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,Wedding Covered but No Honeymoon,10%,481xx,NM,4 years,0
1,1000.0,Consolidating Debt,10%,010xx,MA,< 1 year,0
2,11000.0,Want to consolidate my debt,10%,212xx,MD,1 year,0


Once the data has been imported and column names are the same in both data frames, we add a variable that encodes whether or not the loan has been approved and then merge the data frames. Then we convert non-numerical columns to numerical or categorical ones. I drop the "Loan Title" column (see details in appendix).

In [7]:
declined['Approved'] = 0
approved['Approved'] = 1

loan_data = pd.concat([declined, approved])
print loan_data.shape
loan_data.head(3)

(2270425, 8)


,Amount Requested,Approved,Debt-To-Income Ratio,Employment Length,Loan Title,Policy Code,State,Zip Code
0,1000.0,0,10%,4 years,Wedding Covered but No Honeymoon,0.0,NM,481xx
1,1000.0,0,10%,< 1 year,Consolidating Debt,0.0,MA,010xx
2,11000.0,0,10%,1 year,Want to consolidate my debt,0.0,MD,212xx


## Section 2 - Data Transformation

After initially training a model with a min-max scaled Debt-to-Income Ratio I noticed that this reduced some of the more "normal" DTIs (0.1-10) to insignificant numbers, trumped by a few outstandingly bad scores. 
After having checked the max DTI values in the approved data frame, I found that no loans had ever gotten approved to people with a DTI of more than 40. So I decided to cap the DTI at 50 and round all larger values to that instead.

In [8]:
#this confirms that a dti ratio of over 40 cannot get loans approved
print np.max(approved['Debt-To-Income Ratio'])

#changing the dti ratio to numerical values
loan_data['Debt-To-Income Ratio'] = loan_data['Debt-To-Income Ratio'].str.extract(r'([0-9]+(\.[0-9]+)?)')
loan_data['Debt-To-Income Ratio'] = pd.to_numeric(loan_data['Debt-To-Income Ratio'])

#capping the dti ratio at 50
series = loan_data['Debt-To-Income Ratio']
loan_data['Debt-To-Income Ratio'] = series.apply(lambda x: x if x <= 50.0 else 50.0)
print np.max(loan_data['Debt-To-Income Ratio'])

39.99
50.0


We use regular expressions to extract the numerical values from the text strings.
Since the State is actually reflected in the first two values of the Zip Code, I chose to use Zip as a proxy for State and drop that column instead.
I did consider turning Zip Code into a dummy variable, but simple transformations of that would lead to many unnecessary dimensions for my input data and I could not find simple, clear groupings that would have allowed me to create sub-buckets effectively, without having to make some assumptions about the impact of Zip on loan request. I tested the model with Zip as it was instead, and found it to work sufficiently well to forego the Zip issue.

---
Since all data is now on a roughly similar scale, I chose not to apply any feature scaling. The removed code for it can also be found in the appendix. This choice would also mean, that I will be able to see more clearly what my model coefficients mean.

In [9]:
#extracting numerical values from years worked
print loan_data['Employment Length'].unique()
loan_data['Employment Length'] = loan_data['Employment Length'].replace('< 1 year', '0 year')
loan_data['Employment Length'] = loan_data['Employment Length'].str.extract(r'([0-9]+)')
loan_data['Employment Length'] = pd.to_numeric(loan_data['Employment Length'])

# zipcode -> numerical
loan_data['Zip Code'] = loan_data['Zip Code'].str.extract(r'([0-9]+)')
loan_data['Zip Code'] = pd.to_numeric(loan_data['Zip Code'])

['4 years' '< 1 year' '1 year' '3 years' '2 years' '10+ years' '9 years'
 '5 years' '7 years' '6 years' '8 years' 'n/a' nan]


In [10]:
loan_data = loan_data.drop(['Loan Title', 'State'], axis=1)
loan_data = loan_data.dropna(axis=0, how='any')
print loan_data.shape
loan_data.head(3)

(2225233, 6)


,Amount Requested,Approved,Debt-To-Income Ratio,Employment Length,Policy Code,Zip Code
0,1000.0,0,10.0,4.0,0.0,481.0
1,1000.0,0,10.0,0.0,0.0,10.0
2,11000.0,0,10.0,1.0,0.0,212.0


## Section 3 - Model training & evaluation

We will use a simple Logistic Regression model to predict whether loans were approved or declined. Using grid search, I determine the regularization parameter C to avoid overfitting on the training data. Sag, SAGA and Lbgfs are solvers that are particularly recommended for larger data sets. 

The grid search function also includes cross valiation to further ensure that we are not overfitting on the training data. We perform grid search on a sub-set of the training data (for time constraint reasons) and then train a model with the resulting parameters on all training data before we evaluate it on the test data set.

In [15]:
labels = loan_data['Approved']
predictors = loan_data.drop('Approved', axis=1)

X_train, X_test, y_train, y_test = train_test_split(predictors, labels, test_size=0.4)

#GridSearch on a subsection of the data
X_grid = X_train[:len(X_train)/20]
y_grid = y_train[:len(X_train)/20]

LR = LogisticRegression()
parameters = [{'C': [1.0, 0.1, 0.01], 'max_iter': [500],
              'solver': ['sag', 'saga', 'lbfgs']}]

grid = GridSearchCV(LR, cv=5, param_grid=parameters,
                    scoring='accuracy')
grid.fit(X_grid, y_grid)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'C': [1.0, 0.1, 0.01], 'max_iter': [500], 'solver': ['sag', 'saga', 'lbfgs']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [16]:
print "Parameters used for LR:"
print grid.best_params_
optimal_LR = grid.best_estimator_

#training ideal model on total training data
optimal_LR.fit(X_train, y_train)

print " Coefficients: "
features = predictors.columns.values
coefficients = np.array(optimal_LR.coef_)

for name, coef in zip(features, coefficients[0]):
    print name, 'True', '=', coef
    print name, 'Log', '=', np.log(abs(coef))

Parameters used for LR:
{'C': 1.0, 'max_iter': 500, 'solver': 'lbfgs'}
 Coefficients: 
Amount Requested True = -7.29365690304e-05
Amount Requested Log = -9.52592041194
Debt-To-Income Ratio True = 0.216229342685
Debt-To-Income Ratio Log = -1.53141566289
Employment Length True = 0.232547133503
Employment Length Log = -1.45866234976
Policy Code True = 17.7406056145
Policy Code Log = 2.87585611466
Zip Code True = -0.000936116436095
Zip Code Log = -6.97377069169


In [17]:
y_pred = optimal_LR.predict(X_test)

acc_train = grid.best_score_
acc_test = accuracy_score(y_test, y_pred)
print "LR accuracy training data: ", acc_train
print "LR accuracy testing data: ", acc_test

LR accuracy training data:  1.0
LR accuracy testing data:  0.999992135662


The accuracy scores here indicate that our model is not overfitting and, knowing that the classes are not signficantly unbalanced, we do not have to worry about accuracy mis-representing our true performance.

## Section 3 - Predicting loan request amounts

The below function now takes a customers data and uses it to iteratively predict whether loans get approved, as the amount requested increases. Note, that this assumes, that the relationship between amount requested and approval is linear with a clear cutoff point. Nothing I have seen in the pre-processing led me to believe otherwise, but it would be possible that a person with certain characteristics could, for example, get loans from 500-2000 and 8000-10000, but for some reason not get a 6000 loan. Our model would then stop it's predictions at 2000 already.

The step size of 500 has been chose arbitrarily to shorten prediction time and could be changed if higher precision is desired. 

In [18]:
def how_much_should_I_request():
    request = 0
    
    print "Please enter your information below:"
    debt = int(raw_input("What is your current debt? "))
    income = int(raw_input("What is your annual income? "))
    employ = int(raw_input("How long have you been in your current job? "))
    policy_code = int(raw_input("What's your policy code? (0 or 1) "))
    zip_code = int(raw_input("Please enter the first 3 digits of your zip code: "))
    
    #corrections and errors
    if income <= 0:
        income = 1
    dti = float(debt) / float(income)
    if dti >= 50:
        dti = 50
    
    if employ > 10:
        employ = 10
    
    d = {'Amount': [request], 'Dti': [dti], 'Employment': [employ],
         'Policy': [policy_code], 'Zip': [zip_code]}
    data = pd.DataFrame(d)
    print "You entered the following information: "
    print data.head()
    
    for i in range(500, 40000, 100):
        data['Amount'].iloc[0] = i
        pred = optimal_LR.predict(data)
        if pred == 0:
            print "Here's how much you could request: " + str(data['Amount'].iloc[0] - 100)
            break

In [19]:
how_much_should_I_request()

Please enter your information below:
What is your current debt? 10000
What is your annual income? 30000
How long have you been in your current job? 12
What's your policy code? (0 or 1) 1
Please enter the first 3 digits of your zip code: 432
You entered the following information: 
   Amount       Dti  Employment  Policy  Zip
0       0  0.333333          10       1  432
Here's how much you could request: 16800


---

---

## Appendix

I chose not to pursue the Loan Title column further, since simply vectorizing the text (as in example code below) would have led to extremely high-dimensional data with perhaps little additional predictive value. Instead this text might be useful if the website had chose to create a limited amount of sub-categories for people to chose from. Such categories could also potentially be extraced from the existing text data, but from simply reading through a sub-set of the column, I could not find any apparent categories and hence did not further pursue this avenue.

In [20]:
# tokenizer = nltk.tokenize.regexp.WhitespaceTokenizer()
# lemmatizer = nltk.stem.WordNetLemmatizer()
# vectorizer = TfidfVectorizer()

# def lemmatize(text):
#     text = text.decode('utf-8', errors='ignore')
#     return [lemmatizer.lemmatize(w) for w in tokenizer.tokenize(text)]

# text_data = loan_data['Loan Title']
# text_data = text_data.str.lower()
# text_data = text_data.apply(lemmatize)
# text_data = text_data.apply(vectorizer)

Here is some sample code I was planning on using to scale my values, but chose to apply caps and drop columns instead.

In [21]:
# scaler = MinMaxScaler()

# loan_data_scaled = pd.DataFrame(scaler.fit_transform(loan_data), columns=loan_data.columns)

# loan_data['Debt-To-Income Ratio'] = loan_data_scaled['Debt-To-Income Ratio']
# loan_data['Employment Length'] = loan_data_scaled['Employment Length']
# loan_data['State'] = loan_data_scaled['State']
# loan_data['Zip Code'] = loan_data_scaled['Zip Code']
# loan_data.head()